data was scrapped on june 13 2020

In [1]:
#Import Libraries
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None


## Import and Cleaning

In [2]:
calendar = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/calendar.csv.gz');
listings1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv');
listings2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv.gz');
neighborhoods = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/neighbourhoods.csv');
reviews1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv');
reviews2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv.gz');

#Data Notes:

#locations are not exact (0-450 feet from actual address) due to how airbnb displays data. Therefore a heatmap will work well
#price is in dollars

c:\users\alex cerce\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Start by fixing the data types for each table

In [3]:
#Data cleaning functions used below

#I would prefer integers to have no trailing zeros

#This function will search a specifc dataframe column and return the number of non integer values and NAN values
#I am doing this because I want to keep the trailing zeros if there are values in the array 
#I wont be able to take away trailing zeros if there are any nan values
def find_noninteger(Dataframe,feature): #feature should be input as a string to be consistent with normal pandas format
    value=Dataframe[feature];
    float_count=0;na_count=0;index_nonint=[];index_na=[];
    for i in range(len(value)):
        if (value[i]).is_integer()==0 | pd.isna(value[i])==0:
            float_count +=1
            index_nonint.append(i)
        if pd.isna(value[i])==1:
            na_count +=1
            #index_na.append(i)
    if float_count==1 and na_count==1:
        print('There is',float_count,'non integer value and',na_count,'NAN value')
    elif float_count==1 and na_count!=1:
        print('There is',float_count,'non integer value and',na_count,'NAN values')
    elif float_count!=1 and na_count==1:
        print('There are',float_count,'non integer values and',na_count,'NAN value')
    elif float_count!=1 and na_count!=1:
        print('There are',float_count,'non integer values and',na_count,'NAN values')
    if index_nonint or index_na:
        print('The non integer values are located at:',index_nonint)

### Calendar Data Table
    The calendar table holds availibility data for every listing. The availibility is given from
    the date sampled up to one full year. Therefore, each listing will have 365 sample points.

In [4]:
calendar.head(n=5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,80843,2020-06-14,f,$39.00,$39.00,2.0,1125.0
1,186851,2020-06-15,f,$33.00,$33.00,62.0,300.0
2,186851,2020-06-16,f,$33.00,$33.00,62.0,300.0
3,186851,2020-06-17,f,$33.00,$33.00,62.0,300.0
4,186851,2020-06-18,f,$33.00,$33.00,62.0,300.0


In [5]:
#Missing Values
calendar_missingvalues_df=calendar[calendar.isna().any(axis=1)]    #dataframe containing only rows with any missing values
calendar_missingvalues_columns=calendar.columns[calendar.isna().any()].tolist()  #list of columns with missing values
print('These features have missing values:',calendar_missingvalues_columns)
print('\'minimum_nights\' has',calendar['minimum_nights'].isna().sum(),'missing values')
print('\'maximum_nights\' has',calendar['maximum_nights'].isna().sum(),'missing values')

# 'minimumn_nights' and 'maximum_nights':

#I have found that these 141 missing values are all from different listings, and they all
#occur on the sample date. This is just a data collection error
#I can pull the min and max nights values from other entries since they will be the same for the listing 

#pull the listing_id for the missing value, set missing values to be the same as the following row in full df
#use the sub table to reduce cpu load
for i in range(len(calendar_missingvalues_df)):
    missing_listing_id=calendar_missingvalues_df.iloc[i]['listing_id']
    missing_index=calendar_missingvalues_df.index[calendar_missingvalues_df['listing_id']==missing_listing_id].values[0]
    missing_listing_df=calendar.loc[calendar['listing_id']==missing_listing_id]
    
    calendar.at[missing_index,'minimum_nights']=missing_listing_df.iloc[50]['minimum_nights']  #choose arbitrary future date to set these values to
    calendar.at[missing_index,'maximum_nights']=missing_listing_df.iloc[50]['maximum_nights']
    
print('After filling missing values with the correct values:')
print('\'minimum_nights\' has',calendar['minimum_nights'].isna().sum(),'missing values')
print('\'maximum_nights\' has',calendar['maximum_nights'].isna().sum(),'missing values')


These features have missing values: ['minimum_nights', 'maximum_nights']
'minimum_nights' has 141 missing values
'maximum_nights' has 141 missing values
After filling missing values with the correct values:
'minimum_nights' has 0 missing values
'maximum_nights' has 0 missing values


In [6]:
#check columns for decimal values. This will indicated whether or not I should remove trailing zeros (by converting to INT)
print('minimum nights:')
find_noninteger(calendar,'minimum_nights')
print('maximum nights:')
find_noninteger(calendar,'maximum_nights')
print('-'*50);print('Before:');print(calendar.dtypes)  #This indicates, that once I deal with missing values I can convert these cols to int


#Convert Data Types
#convert to datetime
calendar['date']=calendar['date'].astype('datetime64')
#convert to string dtype and capitalize T/F values
calendar['available']=calendar['available'].str.replace('f','F').str.replace('t','T').astype('str')
#remove dollar signs, remove comas, and convert to float
calendar['price']=calendar['price'].str.replace('$','').str.replace(',','').astype('float64') 
calendar['adjusted_price']=calendar['adjusted_price'].str.replace('$','').str.replace(',','').astype('float64')
#Convert to INT (no decimal place values and no more missing values)
calendar['minimum_nights']=calendar['minimum_nights'].astype(int)
calendar['maximum_nights']=calendar['maximum_nights'].astype(int)
print('-'*50);print('After:')
print(calendar.dtypes)


minimum nights:
There are 0 non integer values and 0 NAN values
maximum nights:
There are 0 non integer values and 0 NAN values
--------------------------------------------------
Before:
listing_id          int64
date               object
available          object
price              object
adjusted_price     object
minimum_nights    float64
maximum_nights    float64
dtype: object
--------------------------------------------------
After:
listing_id                 int64
date              datetime64[ns]
available                 object
price                    float64
adjusted_price           float64
minimum_nights             int32
maximum_nights             int32
dtype: object


In [7]:
calendar.head()

#number of observations in df
#number of unique listings

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,80843,2020-06-14,F,39.0,39.0,2,1125
1,186851,2020-06-15,F,33.0,33.0,62,300
2,186851,2020-06-16,F,33.0,33.0,62,300
3,186851,2020-06-17,F,33.0,33.0,62,300
4,186851,2020-06-18,F,33.0,33.0,62,300


### Listings_1 Data Table

In [8]:
listings1.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ion,Mitte,Brunnenstr. Süd,52.53454,13.40256,Entire home/apt,60,4,132,2020-04-03,2.59,7,180
1,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.53500,13.41758,Entire home/apt,90,62,146,2020-05-27,1.09,1,0
2,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.49885,13.34906,Private room,28,7,27,2019-05-31,0.32,1,364
3,7071,BrightRoom with sunny greenview!,17391,BrightRoom,Pankow,Helmholtzplatz,52.54316,13.41509,Private room,33,1,293,2020-03-31,2.22,2,61
4,9991,Geourgeous flat - outstanding views,33852,Philipp,Pankow,Prenzlauer Berg Südwest,52.53303,13.41605,Entire home/apt,180,6,8,2020-01-04,0.14,1,10


### Listings_2 Data Table

In [9]:
listings2.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2015,https://www.airbnb.com/rooms/2015,20200613063607,2020-06-15,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,...,f,f,strict_14_with_grace_period,f,f,7,7,0,0,2.59
1,3176,https://www.airbnb.com/rooms/3176,20200613063607,2020-06-15,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,none,The neighbourhood is famous for its variety of...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,1.09
2,3309,https://www.airbnb.com/rooms/3309,20200613063607,2020-06-15,BerlinSpot Schöneberg near KaDeWe,First of all: I prefer short-notice bookings. ...,"Your room is really big and has 26 sqm, is ver...",First of all: I prefer short-notice bookings. ...,none,"My flat is in the middle of West-Berlin, direc...",...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,0.32
3,7071,https://www.airbnb.com/rooms/7071,20200613063607,2020-06-15,BrightRoom with sunny greenview!,Cozy and large room in the beautiful district ...,"The BrightRoom is an approx. 20 sqm (215ft²), ...",Cozy and large room in the beautiful district ...,none,"Great neighborhood with plenty of Cafés, Baker...",...,t,f,moderate,f,f,2,0,2,0,2.22
4,9991,https://www.airbnb.com/rooms/9991,20200613063607,2020-06-15,Geourgeous flat - outstanding views,4 bedroom with very large windows and outstand...,"THE APPARTMENT - 4 bedroom (US, Germany: 5 roo...",4 bedroom with very large windows and outstand...,none,Prenzlauer Berg is an amazing neighbourhood wh...,...,f,f,moderate,f,f,1,1,0,0,0.14


### Reviews_1 Data Table

In [10]:
reviews1.head()

,listing_id,date
0,2015,2016-04-11
1,2015,2016-04-15
2,2015,2016-04-26
3,2015,2016-05-10
4,2015,2016-05-14


### Reviews_2 Data Table

In [11]:
reviews2.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2015,69544350,2016-04-11,7178145,Rahel,Mein Freund und ich hatten gute gemütliche vie...
1,2015,69990732,2016-04-15,41944715,Hannah,Jan was very friendly and welcoming host! The ...
2,2015,71605267,2016-04-26,30048708,Victor,Un appartement tres bien situé dans un quartie...
3,2015,73819566,2016-05-10,63697857,Judy,"It is really nice area, food, park, transport ..."
4,2015,74293504,2016-05-14,10414887,Romina,"Buena ubicación, el departamento no está orden..."


### Neighborhoods Data Table

In [13]:
#Table which indicates the borough which each neighborhood belongs to

neighborhoods.columns = ['borough', 'neighborhood']   #rename columns 
neighborhoods.head()

,borough,neighborhood
0,Charlottenburg-Wilm.,Barstraße
1,Charlottenburg-Wilm.,Charlottenburg Nord
2,Charlottenburg-Wilm.,Düsseldorfer Straße
3,Charlottenburg-Wilm.,Forst Grunewald
4,Charlottenburg-Wilm.,Grunewald
